<a href="https://colab.research.google.com/github/TarteTatin1212/contents_search/blob/main/contents_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 必要なライブラリをインポート
import langid
import pandas as pd
import json
import pytz
from datetime import datetime, timedelta
from apiclient.discovery import build
from google.colab import drive


In [ ]:
# Googleドライブのマウント
drive.mount('/content/drive')

# ディレクトリの変更
%cd '/content/drive/MyDrive/Horita_Semi'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Horita_Semi


In [ ]:
%ls

2023年6月1日課題点羅列.gdoc  youtube_data_japanese.csv
api_key.txt                  youtube_data_japanese.json
contents_search.ipynb        youtube_data_other.csv
face_recognition.ipynb       youtube_data_other.json
sample1.jpg                  無題の図形描画.gdraw
test1.jpg                    論文/
test2.JPG                    進捗報告/
Untitled0.ipynb


In [ ]:
# APIキーをテキストファイルから読み込む
with open('api_key.txt', 'r') as file:
    api_key = file.readline().strip()


In [ ]:
from datetime import datetime, timedelta
import pytz

# 日本時間での現在の日付を取得
current_time_japan = datetime.now(pytz.timezone("Asia/Tokyo"))

if current_time_japan.time() < datetime.time(18, 0):
    # 今日の日付から1日引いて、昨日の日付を取得
    target_date = current_time_japan - timedelta(days=1)
else:
    target_date = current_time_japan

# 18:00と21:00の日付オブジェクトを作成
start_time = target_date.replace(hour=18, minute=0, second=0, microsecond=0)
end_time = target_date.replace(hour=21, minute=0, second=0, microsecond=0)

# 日付オブジェクトをISO 8601形式の文字列に変換
start_time_iso = start_time.isoformat()
end_time_iso = end_time.isoformat()


TypeError: ignored

In [ ]:
# 検索リクエストを実行
search_response = youtube.search().list(
    q="",  # 検索クエリ（ここでは全ての動画を対象）
    part="id",  # 動画のIDを取得
    type="video",  # ビデオのみを対象
    publishedAfter=start_time_iso,  # 18:00以降に公開された動画のみを対象
    maxResults=30,  # 最大100件の結果を取得
    regionCode="JP",  # 日本で公開された動画のみを対象
).execute()

print(search_response)


{'kind': 'youtube#searchListResponse', 'etag': 'LgbNAyqK1NqajNz9V3coHms2150', 'regionCode': 'JP', 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 0}, 'items': []}


In [ ]:
# データを保存するリスト
data_japanese = []
data_other = []

# 動画ごとに必要な情報を取得
for item in search_response["items"]:
    video_id = item["id"]["videoId"]

    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part="snippet,statistics",  # 動画の基本情報と視聴回数を取得
        id=video_id,  # 検索で取得した動画IDを指定
    ).execute()
    print("Title:")

    for item in video_response["items"]:
        title = item["snippet"]["title"]  # 動画タイトル
        lang, _ = langid.classify(title)
        channel_title = item["snippet"]["channelTitle"]  # チャンネル名
        thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]  # サムネイルURL
        view_count = item["statistics"]["viewCount"]  # 再生回数

        # 取得した情報を辞書に格納
        video_info = {
        "title": title,
        "channelTitle": channel_title,
        "thumbnailUrl": thumbnail_url,
        "viewCount": view_count
        }

        # データの出力
        print("Title:")
        print("Title:", video_info["title"])
        print("Channel Title:", video_info["channelTitle"])
        print("Thumbnail URL:", video_info["thumbnailUrl"])
        print("View Count:", video_info["viewCount"])


        # タイトルが日本語なら data_japanese に、それ以外なら data_other に追加
        if lang == 'ja':
            data_japanese.append(video_info)
        else:
            data_other.append(video_info)


In [ ]:
 print("Title:", video_info["title"])# JSON形式で保存
with open('youtube_data_japanese.json', 'w') as json_file:
    json.dump(data_japanese, json_file)
with open('youtube_data_other.json', 'w') as json_file:
    json.dump(data_other, json_file)


In [ ]:
# CSV形式で保存
df_japanese = pd.DataFrame(data_japanese)
df_japanese.to_csv('youtube_data_japanese.csv', index=False)
df_other = pd.DataFrame(data_other)
df_other.to_csv('youtube_data_other.csv', index=False)
